In [3]:
# ml.m5.4xlarge
!pip install ImageBind/.
!pip install git+https://github.com/huggingface/transformers.git
!pip install "sagemaker>=2.140.0" "transformers==4.26.1" "datasets[s3]==2.10.1" --upgrade
!pip install accelerate==0.20.3

Processing ./ImageBind
  Preparing metadata (setup.py) ... done
  Using cached pytorchvideo-0.1.5-py3-none-any.whl
  Created wheel for imagebind: filename=imagebind-0.1.0-py3-none-any.whl size=27969 sha256=1e09f37ef3eeb6be2e52220f4cf4266f2c7dfeaccaf849fe99fce998330fa731
  Stored in directory: /tmp/pip-ephem-wheel-cache-j6gk63ba/wheels/15/d5/57/9f60b1256b436b67dceac96201d8bc2eebd9fd320f633e190f
Successfully built imagebind
  Attempting uninstall: imagebind
    Found existing installation: imagebind 0.1.0
    Uninstalling imagebind-0.1.0:
      Successfully uninstalled imagebind-0.1.0

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-f_cjub88
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-f_cjub88
  Resolved https://github.com/huggingface/transformers.git to commit 716bb2e391

In [35]:
from imagebind import data
import torch
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType

import numpy as np
from sagemaker import get_execution_role
import boto3
import pandas as pd
from io import StringIO # Python 3.
from datasets import load_dataset,Dataset,DatasetDict,concatenate_datasets

from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import json

In [16]:
s3 = boto3.resource('s3')
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-ed-demo-2.2/text_repr.json"

# loading in raw data
content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df = pd.DataFrame(json_content).T
print("length of dataframe: "+ str(len(df)))
df.head(5)

length of dataframe: 210


,arrival,eddischarge,admission,discharge,triage,medrecon,vitals,pyxis,codes
37887480,"Patient 10014729, a 21 year old white - other ...",The ED disposition was admitted at 2125-03-19 ...,The patient was admitted at 2125-03-19 16:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...
34176810,"Patient 10018328, a 83 year old white female, ...",The ED disposition was admitted at 2154-02-05 ...,The patient was admitted at 2154-02-05 21:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,NaN,The patient received the following diagnostic ...
32103106,"Patient 10018328, a 83 year old white female, ...",The ED disposition was home at 2154-08-03 22:2...,The patient was not admitted.,The patient was not admitted.,"At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...
38797992,"Patient 10020640, a 91 year old white female, ...",The ED disposition was admitted at 2153-02-13 ...,The patient was admitted at 2153-02-13 00:22:00.,The patient's discharge disposition was: skill...,"At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...
33473053,"Patient 10015272, a 78 year old white female, ...",The ED disposition was admitted at 2137-06-12 ...,The patient was admitted at 2137-06-12 18:36:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...


In [17]:
df['eddischarge'] = [1 if 'admitted' in s.lower() else 0 for s in df['eddischarge']] # admitted = 1, Home = 0
df['medrecon'] = df['medrecon'].fillna("The patient was previously not taking any medications.")
df['pyxis'] = df['pyxis'].fillna("The patient did not receive any medications.")
df['vitals'] = df['vitals'].fillna("The patient had no vitals recorded")
df['codes'] = df['codes'].fillna("The patient received no diagnostic codes")
df = df.drop("admission",axis=1)
df = df.drop("discharge",axis=1)
# df['general'] = df['arrival'] + df['triage'] + df['codes'] + df['pyxis'] 
# df = df.drop('arrival',axis=1)
# df = df.drop('triage',axis=1)
# df = df.drop('codes',axis=1)
# df = df.drop('pyxis',axis=1)
df = df[[col for col in df.columns if col != 'eddischarge'] + ['eddischarge']] # rearrange column to the end
df

,arrival,triage,medrecon,vitals,pyxis,codes,eddischarge
37887480,"Patient 10014729, a 21 year old white - other ...","At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...,1
34176810,"Patient 10018328, a 83 year old white female, ...","At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient did not receive any medications.,The patient received the following diagnostic ...,1
32103106,"Patient 10018328, a 83 year old white female, ...","At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...,0
38797992,"Patient 10020640, a 91 year old white female, ...","At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...,1
33473053,"Patient 10015272, a 78 year old white female, ...","At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...,1
...,...,...,...,...,...,...,...
30272878,"Patient 10038999, a 45 year old white male, ar...","At triage: temperature was not recorded, pulse...",The patient was previously not taking any medi...,The patient had the following vitals: At 2131-...,The patient received the following medications...,The patient received the following diagnostic ...,1
31628990,"Patient 10009049, a 56 year old white male, ar...","At triage: temperature was 99.0, pulse was 87....",The patient was previously taking the followin...,The patient had the following vitals: At 2174-...,The patient received the following medications...,The patient received the following diagnostic ...,1
32405286,"Patient 10004457, a 65 year old white male, ar...","At triage: temperature was 97.6, pulse was 103...",The patient was previously taking the followin...,The patient had the following vitals: At 2141-...,The patient received the following medications...,The patient received the following diagnostic ...,1
34391979,"Patient 10004720, a 61 year old white male, ar...","At triage: temperature was not recorded, pulse...",The patient was previously taking the followin...,The patient had the following vitals: At 2186-...,The patient received the following medications...,The patient received the following diagnostic ...,1


In [21]:
print(df.medrecon.str.len().mean())
print(df.vitals.str.len().mean())
print(df.arrival.str.len().mean())
print(df.triage.str.len().mean())
print(df.pyxis.str.len().mean())
print(df.codes.str.len().mean())

834.6428571428571
982.6333333333333
181.45714285714286
244.43333333333334
278.247619047619
180.56190476190477


In [22]:
medrecon_list = df.medrecon.to_list()
vitals_list = df.vitals.to_list()
arrival_list = df.arrival.to_list()
triage_list = df.triage.to_list()
pyxis_list = df.pyxis.to_list()
codes_list = df.codes.to_list()

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Instantiate model
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)

print("Done Loading Model")

Done Loading Model


In [23]:
with torch.no_grad():
    inputs_medrecon = {ModalityType.TEXT: data.load_and_transform_text(medrecon_list, device)}
    embeddings_medrecon = model(inputs_medrecon)
    inputs_vitals = {ModalityType.TEXT: data.load_and_transform_text(vitals_list, device)}
    embeddings_vitals = model(inputs_vitals)
    inputs_arrival = {ModalityType.TEXT: data.load_and_transform_text(arrival_list, device)}
    embeddings_arrival = model(inputs_general)
    inputs_triage = {ModalityType.TEXT: data.load_and_transform_text(triage_list, device)}
    embeddings_triage = model(inputs_general)
    inputs_pyxis = {ModalityType.TEXT: data.load_and_transform_text(pyxis_list, device)}
    embeddings_pyxis = model(inputs_general)
    inputs_codes = {ModalityType.TEXT: data.load_and_transform_text(codes_list, device)}
    embeddings_codes = model(inputs_general)
    

In [24]:
with torch.no_grad():
    embeddings_medrecon = model(inputs_medrecon)
    embeddings_vitals = model(inputs_vitals)
    embeddings_arrival = model(inputs_arrival)
    embeddings_triage = model(inputs_triage)
    embeddings_pyxis = model(inputs_pyxis)
    embeddings_codes = model(inputs_codes)

In [25]:
f = torch.eq(embeddings_medrecon['text'], embeddings_arrival['text'])
f.unique(return_counts=True)

(tensor([False]), tensor([215040]))

In [26]:
f = torch.eq(embeddings_vitals['text'], embeddings_triage['text'])
f.unique(return_counts=True)

(tensor([False,  True]), tensor([ 48128, 166912]))

In [27]:
f = torch.eq(embeddings_codes['text'], embeddings_medrecon['text'])
f.unique(return_counts=True)

(tensor([False,  True]), tensor([167936,  47104]))

In [28]:
embeddings_codes

{'text': tensor([[-0.5405,  1.5691,  2.6175,  ...,  2.7306,  0.4177,  0.2988],
         [ 0.6689, -4.7472,  0.6694,  ...,  1.7531, -0.3385, -1.5079],
         [ 1.1397, -2.2978, -0.9795,  ..., -0.0491, -0.4359, -3.5429],
         ...,
         [-0.5405,  1.5691,  2.6175,  ...,  2.7306,  0.4177,  0.2988],
         [ 0.3575, -2.7234,  1.5720,  ...,  3.7609, -0.6914, -0.2626],
         [ 0.0445, -1.1765, -1.8933,  ...,  2.9270,  2.3252, -1.5355]])}

In [32]:
print(embeddings_arrival['text'].size())
print(embeddings_vitals['text'].size())
print(embeddings_medrecon['text'].size())
print(embeddings_codes['text'].size())
print(embeddings_pyxis['text'].size())
print(embeddings_triage['text'].size())

torch.Size([210, 1024])
torch.Size([210, 1024])
torch.Size([210, 1024])
torch.Size([210, 1024])
torch.Size([210, 1024])
torch.Size([210, 1024])


In [33]:
# concatenate vectors into one
concat = torch.cat((embeddings_vitals['text'], embeddings_arrival['text'], embeddings_medrecon['text'], embeddings_codes['text'],embeddings_pyxis['text'], embeddings_triage['text']), -1)
print(concat.size())
concat[0]

torch.Size([210, 6144])


tensor([-0.5405,  1.5691,  2.6175,  ...,  0.8123, -0.6005,  1.2162])

In [36]:
class SelfAttention(nn.Module):
    def __init__(self, input_dim):
        super(SelfAttention, self).__init__()
        self.input_dim = input_dim
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)
        self.softmax = nn.Softmax(dim=2)
        
    def forward(self, x):
        queries = self.query(x)
        keys = self.key(x)
        values = self.value(x)
        scores = torch.bmm(queries, keys.transpose(1, 2)) / (self.input_dim ** 0.5)
        attention = self.softmax(scores)
        weighted = torch.bmm(attention, values)
        return weighted

In [42]:
self_attention = SelfAttention( 6144 )
output = self_attention(concat.unsqueeze(0))
output = output[0]

torch.Size([210, 6144])

In [43]:
# apply a dense layer to project them on different space.
dense_layer = nn.Linear(6144, 768)
new_vec = dense_layer(concat)
print(new_vec.size())

torch.Size([210, 768])
